In [9]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("Auto_Reply_v3.db")

# Load Q&A pairs (assuming table name is 'All_Reviews')
query1 = 'SELECT "translation" AS Review, "Developer Reply Text" AS Reply FROM All_Reviews'
query2 = 'SELECT "translation" AS Review, "Reply" FROM "Auto Reply Hadith"'
query3 = 'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews'
query4 = 'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews_Al_Quran'

df1 = pd.read_sql_query(query1, conn)
df2 = pd.read_sql_query(query2, conn)
df3 = pd.read_sql_query(query3, conn)
df4 = pd.read_sql_query(query4, conn)

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Close connection
conn.close()

# Check data
df.head()

# craete a csv file 
# df.to_csv("All_Review_Reply.csv", index=False)

,Review,Reply
0,"Alhamdulillah.. the best al quran application,...",None
1,the best quraan app you can find. it literally...,None
2,Very helpful,None
3,The recent edition does explain the source of ...,BarakAllahu feekum for your kind words and sup...
4,May Allah reward you richly rewarded,None


In [10]:
import torch
from transformers import pipeline

In [11]:
# Check CUDA availability and set device
cuda_available = torch.cuda.is_available()
if cuda_available:
    device = 0  # GPU
    gpu_name = torch.cuda.get_device_name(device)
    print(f"CUDA is available. Using GPU: {gpu_name}")
else:
    device = -1  # CPU
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU: NVIDIA GeForce RTX 4070 Ti SUPER


In [12]:
# Initialize sentiment-analysis pipeline (uses SST-2 fine-tuned DistilBERT)
sentiment_analyzer = pipeline(
    'sentiment-analysis',
    model='distilbert-base-uncased-finetuned-sst-2-english',
    device=device
)

# Function to analyze sentiment (truncates to 512 chars to respect token limits)
def analyze_sentiment(text):
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return None
    snippet = text[:512]
    result = sentiment_analyzer(snippet)
    return result[0]['label']

# Apply sentiment analysis to the 'Review' column
df['Sentiment'] = df['Review'].apply(analyze_sentiment)

df


Device set to use cuda:0


,Review,Reply,Sentiment
0,"Alhamdulillah.. the best al quran application,...",None,POSITIVE
1,the best quraan app you can find. it literally...,None,POSITIVE
2,Very helpful,None,POSITIVE
3,The recent edition does explain the source of ...,BarakAllahu feekum for your kind words and sup...,POSITIVE
4,May Allah reward you richly rewarded,None,POSITIVE
...,...,...,...
70455,\tVery good app,BarakAllahu feekum for your kind words and sup...,POSITIVE
70456,"\tThank God, this application is really good!!...",BarakAllahu feekum for your kind words and sup...,POSITIVE
70457,"\tMashallah, may God bless you",BarakAllahu feekum for your kind words and sup...,POSITIVE
70458,\tThank God this application helps👍,BarakAllahu feekum for your kind words and sup...,POSITIVE


In [ ]:
# 5. Generate automated replies based on sentiment

def generate_reply(sentiment):
    if sentiment == "positive":
        return (
            "BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!"
        )
    elif sentiment == "negative":
        return (
            "Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feedback.gtaf.org. JazakAllahu khayran."
        )
    else :
        return "JazakAllahu khayran for your review! If you face any issues or have feature requests, please provide your feedback on feedback.gtaf.org."

# Apply reply generation
df['Reply'] = df['sentiment'].apply(generate_reply)

# 6. Save the results to CSV including Review, sentiment, and reply
df.to_csv('1.distilbert.csv', index=False)
print(f"Processed {len(df)} reviews. Results saved to '1.distilbert.csv'.")

Processed 70460 reviews. Results saved to '1.distilbert.csv'.
